# LQR Control Lab. Helper - Calculating Different K Matrices Depending On Q and R

In [8]:
import numpy as np
from scipy.linalg import solve_continuous_are
import matplotlib.pyplot as plt
# !pip install control # Run this if control library is not installed, otherwise use scipy
import control as ct

Here are the system parameters to define the Linearized model of the system:

In [9]:
# Define the physical constants

## Motor
# Resistance
Rm = 8.4
# Current-torque (N-m/A)
kt = 0.042
# Back-emf constant (V-s/rad)
km = 0.042
#
## Rotary Arm
# Mass (kg)
mr = 0.095
# Total length (m)
r = 0.085
# Moment of inertia about pivot (kg-m^2)
Jr = mr*r**2/3
# Equivalent Viscous Damping Coefficient (N-m-s/rad)
br = 1e-3 # damping tuned heuristically to match QUBE-Sero 2 response
#
## Pendulum Link
# Mass (kg)
mp = 0.024
# Total length (m)
Lp = 0.129
# Pendulum center of mass (m)
l = Lp/2
# Moment of inertia about pivot (kg-m^2)
Jp = mp*Lp**2/3
# Equivalent Viscous Damping Coefficient (N-m-s/rad)
bp = 5e-5 # damping tuned heuristically to match QUBE-Sero 2 response
# Gravity Constant
g = 9.81

# Find Total Inertia
Jt = Jr*Jp - mp**2*r**2*l**2

Let's create a linearized model of the system. The state-space representation of the system is given below:

In [ ]:
A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, mp**2*l**2*r*g/Jt, -br*Jp/Jt, -mp*l*r*bp/Jt],
    [0, mp*g*l*Jr/Jt, -mp*l*r*br/Jt, -Jp*bp/Jt]
])

B = np.array([
    [0],
    [0],
    [Jp/Jt],
    [mp*l*r/Jt]
])

C = np.eye(2, 4)

D = np.zeros((2, 1))

# Add actuator dynamics
# Transform torque into motor voltage
A[2,2] = A[2,2] - km*km/Rm*B[2]
A[3,2] = A[3,2] - km*km/Rm*B[3]
B = km * B / Rm

sys = ct.ss(A, B, C, D)

Let's define Q and R matrices and calculate the K matrix for different Q and R matrices.

In [ ]:
# Let's call this as standart gain matrix K, penalizes every state variable equally

Q = np.eye(4)
R = 1

K, _, _ = ct.lqr(sys, Q, R)
print("K", K)

In [ ]:

Q = np.eye(4)
Q[0,0] = 20
# Since we are penalizing theta more, the system will try to keep theta as close to zero as possible
R = 1


K_theta_high, _, _ = ct.lqr(sys, Q, R)
print("K_theta_high = ", K_theta_high)

In [ ]:
Q = np.eye(4)
Q[1,1]= 20
# Since we are penalizing alpha more, the system will try to keep alpha as close to zero as possible
R = 1

K_alpha_high, _, _ = ct.lqr(sys, Q, R)
print("K_alpha_high = ", K_alpha_high)

In [ ]:
Q = np.eye(4)
Q[0,0] = 0.1
# Since we are penalizing theta less, the system won't care much about theta
R = 1

K_theta_low, _, _ = ct.lqr(sys, Q, R)
print("K_theta_low = ", K_theta_low)